<a href="https://colab.research.google.com/github/zegabr/pln-chatbot/blob/main/notebooks/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
intent_mapper = {
  'NEGATE': 0,
  'NEGATE_INTENT': 1,
  'REQUEST_ALTS': 2,
  'GOODBYE': 3,
  'REQUEST': 4,
  'THANK_YOU': 5,
  'AFFIRM': 6,
  'AFFIRM_INTENT': 7,
  'SELECT': 8,
  'INFORM': 9,
  'INFORM_INTENT': 10
}

In [3]:
TRAIN_DATASET_PATH = 'https://raw.githubusercontent.com/zegabr/pln-chatbot/main/dataset/classification/train_dataset.csv'
TEST_DATASET_PATH = 'https://raw.githubusercontent.com/zegabr/pln-chatbot/main/dataset/classification/test_dataset.csv'

In [4]:
from pandas import read_csv
train_restaurant_dataset = read_csv(TRAIN_DATASET_PATH)
# pega 2 np arrays, um com as frases e outro com os respectivos intents
train_dataset = train_restaurant_dataset.drop_duplicates(subset=['Phrase'])
train_phrases = np.array(train_restaurant_dataset.Phrase)[1:]
train_intents = np.array(train_restaurant_dataset.Intent.map(lambda x : intent_mapper[x]))[1:]
print(train_phrases)
print(train_intents)

['It has to be in San Fran.' "It'll be afternoon 12." "I'd like Palmer's."
 ... 'That sounds great! Thank you.' 'No, thank you very much!'
 'No, thank you very much!']
[9 9 9 ... 5 5 0]


In [23]:
test_dataset = read_csv(TEST_DATASET_PATH)
# pega 2 np arrays, um com as frases e outro com os respectivos intents
test_dataset = test_dataset.drop_duplicates(subset=['Phrase'])
test_phrases = np.array(test_dataset.Phrase)[1:]
test_intents = np.array(test_dataset.Intent.map(lambda x : intent_mapper[x]))[1:]
print(test_phrases[:10])
print(test_intents[:10])

["Check in the San Jose area. I'd like to find a place that serves Diner style food."
 'Great! can you make a reservation for one on Saturday this week?'
 'For lunch at 12 in the afternoon.'
 'On second thought, make it for Tuesday next week.'
 'Yes. Also see if they have outdoor seating and what their rating is.'
 'Great! Thank you.' "No, thank you. I'm good for now."
 'Can you help me find a place to eat?' "I'm interested in Napa."
 "Yes, I'm interested in Japanese."]
[9 8 9 0 4 5 5 9 9 9]


In [6]:
VOCAB_SIZE = 2000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_phrases)

In [7]:
vocab = np.array(encoder.get_vocabulary())
print(len(encoder.get_vocabulary()))
vocab[:20]

1180


array(['', '[UNK]', 'i', 'for', 'the', 'a', 'to', 'you', 'that', 'is',
       'in', 'yes', 'make', 'it', 'at', 'reservation', 'me', 'restaurant',
       'no', 'thats'], dtype='<U15')

In [8]:
model = tf.keras.Sequential([
  encoder,
  tf.keras.layers.Embedding(
    input_dim=len(encoder.get_vocabulary()),
    output_dim=64,
    # Use masking to handle the variable sequence lengths
    mask_zero=True),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
  tf.keras.layers.Dense(11, activation='softmax')
])

In [9]:
# predict on a sample text without padding.

sample_text = ('Alright, fine. Now I would like to find a one-way flight to go '
               'there, and I do not mind which airline I will have.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.09021422 0.09152975 0.09125305 0.0927147  0.09135037 0.09156321
 0.09007676 0.09109601 0.09083001 0.09063284 0.08873904]


In [10]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[0.09021422 0.09152975 0.09125305 0.0927147  0.09135037 0.09156321
 0.09007675 0.09109601 0.09083001 0.09063284 0.08873904]


In [11]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [12]:
history = model.fit(train_phrases, train_intents, epochs=20)

Epoch 1/20
112/112 [==============================] - 11s 34ms/step - loss: 2.3281 - accuracy: 0.3864
Epoch 2/20
112/112 [==============================] - 4s 34ms/step - loss: 1.9201 - accuracy: 0.4102
Epoch 3/20
112/112 [==============================] - 4s 34ms/step - loss: 1.7042 - accuracy: 0.4335
Epoch 4/20
112/112 [==============================] - 4s 33ms/step - loss: 1.5392 - accuracy: 0.5241
Epoch 5/20
112/112 [==============================] - 4s 33ms/step - loss: 1.3802 - accuracy: 0.5946
Epoch 6/20
112/112 [==============================] - 4s 34ms/step - loss: 1.2643 - accuracy: 0.6257
Epoch 7/20
112/112 [==============================] - 4s 33ms/step - loss: 1.1700 - accuracy: 0.6529
Epoch 8/20
112/112 [==============================] - 4s 33ms/step - loss: 1.0801 - accuracy: 0.6664
Epoch 9/20
112/112 [==============================] - 4s 34ms/step - loss: 0.9956 - accuracy: 0.6726
Epoch 10/20
112/112 [==============================] - 4s 33ms/step - loss: 0.9289 - accur

In [17]:
sample_test = ('I\'m looking for a restaurant, can you help?')
sample_test_intent = 'INFORM'
predictions = model.predict(np.array([sample_test]))
for i in predictions:
  for idx, intent in enumerate(i):
    print('{:.2f}%'.format(intent*100), [i for i in intent_mapper.keys()][idx])
print('uhu?')

3.79% NEGATE
0.00% NEGATE_INTENT
1.14% REQUEST_ALTS
0.00% GOODBYE
0.01% REQUEST
0.02% THANK_YOU
0.07% AFFIRM
0.00% AFFIRM_INTENT
0.43% SELECT
94.53% INFORM
0.00% INFORM_INTENT
uhu?


In [21]:
sample_test = ('Also see if they have outdoor seating and what their rating is')
sample_test_intent = 'REQUEST'
predictions = model.predict(np.array([sample_test]))
for i in predictions:
  for idx, intent in enumerate(i):
    print('{:.2f}%'.format(intent*100), [i for i in intent_mapper.keys()][idx])
print('uhu?')

0.00% NEGATE
0.00% NEGATE_INTENT
0.11% REQUEST_ALTS
0.01% GOODBYE
83.91% REQUEST
0.01% THANK_YOU
15.64% AFFIRM
0.00% AFFIRM_INTENT
0.21% SELECT
0.09% INFORM
0.01% INFORM_INTENT
uhu?
